# Loading data

In [65]:
import pandas as pd

## Loading metrics

In [66]:
androidListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\androidSamples.csv")
awsListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\awsSamples.csv")
azureListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\azureSamples.csv")
springListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\springSamples.csv")

In [67]:
len(androidListSamples)

84

In [68]:
len(awsListSamples)

8

In [69]:
len(azureListSamples)

31

In [70]:
len(springListSamples)

36

In [71]:
len(androidListSamples) + len(awsListSamples) + len(azureListSamples) + len(springListSamples)

159

### 159 Code Samples

In [72]:
def dealWithData(listSamples):    
    listSamples["windowsPath"] = listSamples["path"].str.replace("/", "\\")
    
    allSamples = [pd.read_csv("..\\..\\2-ExtractingMetrics\\metrics\\"+sample+".csv", parse_dates=True) for sample in listSamples["windowsPath"]]

    allSamplesMetrics = pd.concat(allSamples, axis=0, ignore_index=True)

    allSamplesMetrics['commitDate'] = pd.to_datetime(allSamplesMetrics['commitDate'].astype(str).str[:-6])

    allSamplesMetrics.sort_values(by="commitDate", inplace=True)
    
    print(allSamplesMetrics.shape)
        
    return allSamplesMetrics

In [73]:
androidSamples = dealWithData(androidListSamples)
awsSamples = dealWithData(awsListSamples)
azureSamples = dealWithData(azureListSamples)
springSamples = dealWithData(springListSamples)

(14139, 65)
(723, 65)
(2799, 65)
(18708, 65)


In [74]:
(androidSamples.shape[0] + awsSamples.shape[0] +  azureSamples.shape[0] +  springSamples.shape[0])/3

12123.0

In [75]:
def addOtherMetrics(dataframeDestiny, dataframeSource):
    dataframeDestiny["numberJavaFiles"] = dataframeSource[dataframeSource["type"] == "absolute"]["numberJavaFiles"].values
    dataframeDestiny["CountDeclClass"] = dataframeSource[dataframeSource["type"] == "absolute"]["CountDeclClass"].values

Getting only average

In [76]:
androidSamplesAverage = androidSamples[androidSamples["type"] == "average"]
addOtherMetrics(androidSamplesAverage, androidSamples)

awsSamplesAverage = awsSamples[awsSamples["type"] == "average"]
addOtherMetrics(awsSamplesAverage, awsSamples)

azureSamplesAverage = azureSamples[azureSamples["type"] == "average"]
addOtherMetrics(azureSamplesAverage, azureSamples)

springSamplesAverage = springSamples[springSamples["type"] == "average"]
addOtherMetrics(springSamplesAverage, springSamples)

<ipython-input-75-f7169c94c265>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframeDestiny["numberJavaFiles"] = dataframeSource[dataframeSource["type"] == "absolute"]["numberJavaFiles"].values
<ipython-input-75-f7169c94c265>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframeDestiny["CountDeclClass"] = dataframeSource[dataframeSource["type"] == "absolute"]["CountDeclClass"].values


In [77]:
def transformReadabilityPerFileToReadabilityPerClass(dataframe):
    dataframe["readability"] = (dataframe["readability"] * dataframe["numberJavaFiles"]) / dataframe["CountDeclClass"]

In [78]:
transformReadabilityPerFileToReadabilityPerClass(androidSamplesAverage)
transformReadabilityPerFileToReadabilityPerClass(awsSamplesAverage)
transformReadabilityPerFileToReadabilityPerClass(azureSamplesAverage)
transformReadabilityPerFileToReadabilityPerClass(springSamplesAverage)

<ipython-input-77-6fba051f18ca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["readability"] = (dataframe["readability"] * dataframe["numberJavaFiles"]) / dataframe["CountDeclClass"]


In [79]:
def generateStats(dataframe, framework):
    stats = pd.DataFrame(index = dataframe.min().index)
    stats["min"] = dataframe.min()
    stats["max"] = dataframe.max()
    stats["avg"] = dataframe.mean()
    stats["med"] = dataframe.median()
    stats["std"] = dataframe.std()
    stats.to_csv(framework+"_stats.csv")

In [80]:
generateStats(androidSamplesAverage, "android")
generateStats(awsSamplesAverage, "aws")
generateStats(azureSamplesAverage, "azure")
generateStats(springSamplesAverage, "spring")

Questions

In [81]:
def groupBy(dataframe):
    dataframe.set_index(dataframe["commitDate"], inplace=True)
    del dataframe["commitDate"]
    return dataframe.groupby([(dataframe.index.year), (dataframe.index.month)]).mean()

In [82]:
androidMetrics = groupBy(androidSamplesAverage)
awsMetrics = groupBy(awsSamplesAverage)
azureMetrics = groupBy(azureSamplesAverage)
springMetrics = groupBy(springSamplesAverage)

## Loading SO questions

In [83]:
androidQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\androidQuestions.csv")
awsQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\awsQuestions.csv")
azureQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\azureQuestions.csv")
springQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\springQuestions.csv")

In [84]:
def groupBy(dataframe):
    dataframe['creationDate'] = pd.to_datetime(dataframe['creationDate'])
    dataframe.sort_values(by="creationDate", inplace=True)
    dataframe.set_index(dataframe['creationDate'], inplace=True)
    dataframe["questions"] = 1
    del dataframe["ownerUserId"]
    del dataframe["postTypeId"]
    del dataframe["acceptedAnswerId"]
    del dataframe["id"]
    return dataframe.groupby([(dataframe.index.year), (dataframe.index.month)]).sum()

In [85]:
androidQuestions = groupBy(androidQuestions)
awsQuestions = groupBy(awsQuestions)
azureQuestions = groupBy(azureQuestions)
springQuestions = groupBy(springQuestions)

## Merging the data

In [86]:
def merging(dataframe1, dataframe2):
    df = pd.concat([dataframe1, dataframe2], axis=1)
    df["questions"].fillna(0, inplace=True)
    df.fillna(method="ffill", inplace=True)
    return df

In [87]:
android = merging(androidMetrics, androidQuestions)
aws = merging(awsMetrics, awsQuestions)
azure = merging(azureMetrics, azureQuestions)
spring = merging(springMetrics, springQuestions)

## Saving

In [91]:
def save(dataframe, framework):
    dataframe.to_csv(framework+".csv")

In [92]:
save(android, "android")
save(aws, "aws")
save(azure, "azure")
save(spring, "spring")